In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

2025-01-18 19:31:12.513249: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-18 19:31:12.522097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737221472.531806   58860 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737221472.534873   58860 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-18 19:31:12.545680: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
# I need to split train data into labels and features
train_labels = train_data['label'].to_numpy()
train_features = train_data['text'].to_numpy()

In [4]:
test_labels = test_data['label'].to_numpy()
test_features = test_data['text'].to_numpy()

In [5]:
#remove stopwords from train_features
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

for i in range(len(train_features)):
    word_tokens = train_features[i].split()
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    train_features[i] = ' '.join(filtered_sentence)

for i in range(len(test_features)):
    word_tokens = test_features[i].split()
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    test_features[i] = ' '.join(filtered_sentence)

print(train_features)
print(test_features)

['I\'m brand-new forums. I\'ve come seek guidance/wisdom suffered currently suffering depression. Within past 3 years I struggled disease. For part I\'ve control, relatively recently. About four months ago I drastically descending downwards mental health wise. The result declination helped obstruct aspects social life, causing abrupt, painful end many friendships I endeared. Also negative behavior started poison. The equilibrium family I shared. The result lead choice isolation , even I chose isolate myself, I still damaged relationships much, attempts make amends, repair former friendships, futile. I basically pushed alienated everything I held dearest. Anyways, one day I snapped; ability process information, perform simple math equations, remember things I JUST thought seconds before, concentrate, quick wit, confident decision making impaired immensely. Previously, this, I "A" Honor Role student difficulty functioning efficiently "AP" classes, I barely sustain low "B" average. Has an

[nltk_data] Downloading package stopwords to /home/cat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/cat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
for i in range(len(train_features)):
    train_features[i] = train_features[i].lower()[:200]
for i in range(len(test_features)):
    test_features[i] = test_features[i].lower()[:200]
tokenizer.fit_on_texts(train_features)
word_index = tokenizer.word_index
print(word_index)

train_sequences = tokenizer.texts_to_sequences(train_features)
train_padded = pad_sequences(train_sequences, padding='post').astype(np.float32)

test_sequences = tokenizer.texts_to_sequences(test_features)
test_padded = pad_sequences(test_sequences, padding='post').astype(np.float32)

{'<OOV>': 1, 'i': 2, "i'm": 3, 'like': 4, 'my': 5, 'know': 6, 'years': 7, "i've": 8, 'it': 9, 'really': 10, 'time': 11, 'feel': 12, 'one': 13, 'get': 14, 'me': 15, 'so': 16, 'depression': 17, 'friends': 18, 'want': 19, 'breast': 20, 'friend': 21, 'year': 22, 'life': 23, 'got': 24, 'this': 25, 'would': 26, 'going': 27, 'cancer': 28, 'people': 29, 'go': 30, 'think': 31, 'old': 32, 'well': 33, 'the': 34, 'last': 35, 'first': 36, 'new': 37, 'family': 38, 'ago': 39, '2': 40, 'good': 41, 'much': 42, 'day': 43, 'things': 44, 'since': 45, 'back': 46, 'two': 47, 'even': 48, 'i’m': 49, 'we': 50, 'right': 51, 'anyone': 52, 'work': 53, 'see': 54, 'today': 55, 'never': 56, 'went': 57, 'school': 58, 'need': 59, 'now': 60, 'she': 61, 'diagnosed': 62, 'best': 63, "can't": 64, 'months': 65, 'everyone': 66, 'help': 67, 'something': 68, 'always': 69, 'hi': 70, 'started': 71, 'but': 72, 'recently': 73, 'make': 74, 'still': 75, 'mom': 76, 'getting': 77, 'found': 78, '3': 79, 'a': 80, 'long': 81, 'and': 82,

In [7]:
#print distinct labels
print(np.unique(train_labels))
print(np.unique(test_labels))

# convert labels to distinct integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_labels)
train_labels = label_encoder.transform(train_labels)
test_labels = label_encoder.transform(test_labels)

['bc' 'do' 'ds' 'dsf' 'ff' 'nd' 'ndf']
['do' 'nd']


In [8]:
print(train_padded.shape)
#pad train_padded to have same shape as test_padded
test_padded = np.pad(test_padded, ((0,0), (0, train_padded.shape[1] - test_padded.shape[1])), 'constant')
print(test_padded.shape)
print(train_padded.shape)

(2800, 45)
(800, 45)
(2800, 45)


In [9]:
#normalize the data
# train_padded = train_padded / 10000
# test_padded = test_padded / 10000

In [ ]:
print(train_padded)
print(train_labels)
print(test_labels)

In [12]:
# From train data select only those entries that have label 1 or 5
# train_padded = train_padded[np.where((train_labels == 1) | (train_labels == 5))]
# train_labels = train_labels[np.where((train_labels == 1) | (train_labels == 5))]

In [ ]:
#Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')  # Or adjust for multiclass
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, 25))
model.summary()


In [9]:
# LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, 25))
model.summary()


I0000 00:00:1737221483.073493   58860 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13512 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 25, 64)         │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 25, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 688,007 (2.62 MB)

 Trainable params: 688,007 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the model
history = model.fit(train_padded, train_labels, epochs=50, validation_data=(test_padded, test_labels), verbose=2)


Epoch 1/50


I0000 00:00:1737221489.106723   58977 cuda_dnn.cc:529] Loaded cuDNN version 90300


88/88 - 4s - 44ms/step - accuracy: 0.2004 - loss: 1.8520 - val_accuracy: 0.3900 - val_loss: 1.7120
Epoch 2/50
88/88 - 2s - 19ms/step - accuracy: 0.3496 - loss: 1.5078 - val_accuracy: 0.4825 - val_loss: 2.8689
Epoch 3/50
88/88 - 2s - 19ms/step - accuracy: 0.4614 - loss: 1.2713 - val_accuracy: 0.4863 - val_loss: 3.2711
Epoch 4/50
88/88 - 2s - 18ms/step - accuracy: 0.5446 - loss: 1.0942 - val_accuracy: 0.4787 - val_loss: 3.7222
Epoch 5/50
88/88 - 2s - 20ms/step - accuracy: 0.5900 - loss: 0.9910 - val_accuracy: 0.4725 - val_loss: 2.8789
Epoch 6/50
88/88 - 2s - 20ms/step - accuracy: 0.6343 - loss: 0.8781 - val_accuracy: 0.4712 - val_loss: 3.3682
Epoch 7/50
88/88 - 2s - 20ms/step - accuracy: 0.7046 - loss: 0.7623 - val_accuracy: 0.4225 - val_loss: 3.2275
Epoch 8/50
88/88 - 2s - 20ms/step - accuracy: 0.7411 - loss: 0.6964 - val_accuracy: 0.4725 - val_loss: 3.9444
Epoch 9/50
88/88 - 2s - 18ms/step - accuracy: 0.8000 - loss: 0.5980 - val_accuracy: 0.4787 - val_loss: 3.5039
Epoch 10/50
88/88 - 2

In [11]:
# Test the model on test data
test_loss, test_accuracy = model.evaluate(test_padded, test_labels)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7657 - loss: 3.5894
Test loss: 8.8139009475708, Test accuracy: 0.47749999165534973
